In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
from scipy.stats import norm
from scipy.stats import multivariate_normal

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Layer, Dropout, BatchNormalization, LeakyReLU, Lambda
from keras.losses import Loss, mse, MeanSquaredError
from keras.optimizers import Optimizer, Adam
from keras.metrics import Mean
from keras.utils import to_categorical, plot_model, load_img, img_to_array
from keras.callbacks import EarlyStopping
from keras.models import load_model

import tensorflow_probability as tfp
import panel as pn
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Latex, Markdown

np.random.seed(1234)
tf.random.set_seed(1234)
pn.extension('katex', 'mathjax', 'plotly')

# Bayesian Neural Network (BNN) - Theory

The objective of BNNs is to determine the posterior distribution $p( \mathbf{w}| \mathbf{D})$ that best reflects the optimal weights given the data. The key lies in understanding the concepts of the "prior" and the "posterior". The prior belief,$p(\mathbf{w})$, represents our initial hypothesis about the weights $\mathbf{w}$. As new data $\mathbf{D}$ becomes available, we update this belief to form a posterior belief,$p(\mathbf{w}| \mathbf{D})$. The theorem is formulated as:


$$p(\mathbf{w}| \mathbf{D}) = \frac{p(\mathbf{w})p(\mathbf{D}|\mathbf{w})}{p(\mathbf{D})}$$

Each term in this equation is defined as follows:

- $p( \mathbf{w})$: The prior probability of the weights$\mathbf{w}$before observing the data $D$.

- $p( \mathbf{w} | \mathbf{D})$: The posterior probability of the weights $\mathbf{w}$   after observing the data$\mathbf{D}$.

- $p( \mathbf{D} |\mathbf{w})$: The likelihood of observing the data$D$given the weights $\mathbf{w}$.

- $p( \mathbf{D})$: The total probability or marginal likelihood of observing the data $\mathbf{D}$ under all possible weights $\mathbf{w}$.

# Feedforward Step

To illustrate the feedforward step in a Bayesian Neural Network (BNN), consider a BNN with an input layer denoted by the vector $\mathbf{x}$ consisting of $m$ neurons, a hidden layer vector represented as $\mathbf{z}$ with $n$ neurons, and an output layer vector $\mathbf{y}$ . The architecture ensures that every neuron in the input layer is connected to every neuron in the hidden layer through a set of weights, resulting in a fully connected network with three layers. Consider the following image to visualize this architecture:

<center><img src="Images/bnn_ff.svg" width="800" height="800"></center>

This image highlights the connections of the weights and biases from the input layer to a single neuron in the hidden layer, and from this neuron to the output neurons of the final layer.

For a BNN, weights and biases are not fixed values. Instead, each weight and bias  are associated with a mean and a variance, which are parameters of the Gaussian distribution from which they are sampled at each forward step. This introduces variability in the weights and biases, allowing BNNs to inherently model uncertainty. The reparameterization trick is employed to effectively sample the weight and bias values from their respective independent Gaussian distributions and to guarantee differentiability during the backpropagation step. To further elucidate, consider the weights and biases between the input and hidden layers:

$$ w_{ij} = \mu_{ij} + \sigma_{ij} \cdot \epsilon_{ij} $$
$$ b_{j} = \mu_{j} + \sigma_{j} \cdot \epsilon_{j} $$

Here, each $w_{ij}$ connects the $i$-th neuron in the input layer to the $j$-th neuron in the hidden layer, while $b_j$ is the bias for the $j$-th neuron in the hidden layer. $\epsilon_{ij}$ and $\epsilon_{j}$ are random samples from standard normal distributions $\mathcal{N}(0, 1)$. The terms $w_{ij}$ and $b_j$ are recognized as latent variables, allowing us to treat the weights and biases as independent variables sampled from specific distributions. In matrix notation, the weight matrix and bias vector connecting the layers $\mathbf{x}$ and $\mathbf{z}$ can be represented as:

$$ 
\mathbf{w}_z =
\left( \begin{array}{cccc}
w_{11} & \cdots & w_{1n}\\
\vdots & \ddots & \vdots\\
w_{m1} & \cdots &w_{mn}
\end{array} \right),~~~~
\mathbf{b}_z =
\left( \begin{array}{c}
b_{1}\\
\vdots\\
b_{n}
\end{array} \right)
$$

Now, applying the parametrization trick in matrix form for the weights between these layers, we get:


$$ \mathbf{w}_z = {\mu}_w + {\Sigma}_w \odot {\epsilon}_w $$
$$ \mathbf{b}_z = {\mu}_b +{\Sigma}_b \odot {\epsilon}_b $$


The mean and variance matrices and vectors for the weights and biases are typically initialized by sampling random values from a truncated normal distribution, represented as $\mathcal{T}\left(0, \sqrt{\frac{2}{n+m}}\right)$. This particular distribution ensures that the initialized weights and biases aren't too distant from the mean, as extreme values might negatively affect the learning process. The matrix representation for the mean and variance of weights and biases are:

$$ 
{\mu}_w =
\left( \begin{array}{cccc}
\mu_{11} & \cdots & \mu_{1n}\\
\vdots & \ddots & \vdots\\
\mu_{m1} & \cdots &\mu_{mn}
\end{array} \right)~~~~
{\sigma}_w =
\left( \begin{array}{cccc}
\sigma_{11} & \cdots & \sigma_{1n}\\
\vdots & \ddots & \vdots\\
\sigma_{m1} & \cdots &\sigma_{mn}
\end{array} \right).
$$

and

$$ 
{\mu}_b =
\left( \begin{array}{c}
\mu_{1} \\
\vdots \\
\mu_{n}
\end{array} \right)~~~~
{\sigma}_b =
\left( \begin{array}{c}
\sigma_{1}\\
\vdots\\
\sigma_{n}
\end{array} \right).
$$


While each weight and bias is modeled by an independent distribution, it’s important to note that, once sampled, they will influence each other through the network's architecture and the interactions during the feedforward and backpropagation steps. The reparameterization trick is utilized to allow gradients to pass through the sampling process, enabling the learning of the distribution's parameters.

Starting the feedforward process, the weighted sum of the inputs is computed, the bias $\mathbf{b}$ is added, and the result is then passed through an activation function $f(.)$. The equations that describe this process are:


$$
\left\{ \begin{array}{ll} 
\mathbf{\textrm{net}_z} = \mathbf{x} \cdot \mathbf{w}_z+ \mathbf{b}_z\\
\mathbf{z} = f \left(\mathbf{\textrm{net}_z}\right)
\end{array}\right.
$$

for the connection between the input and hidden layers and 


$$
\left\{ \begin{array}{ll} 
\mathbf{\textrm{net}_y} = \mathbf{z}\cdot \mathbf{w}_y+ \mathbf{b}_y\\
\mathbf{y} = f \left(\mathbf{\textrm{net}_y}\right)
\end{array}\right.
$$

for the connection between the hidden and output layers.

In [ ]:
# Test for the Xavier initializer with truncated normal distribution
def xavier(n_in, n_out):
    return tf.random.truncated_normal(
        shape=(n_in, n_out), 
        mean=0.0,
        stddev=np.sqrt(2.0 / (n_in + n_out)))


n_in, n_out = [2, 2] 
w_mean = xavier(n_in, n_out)
w_std = xavier(n_in, n_out) - 6

fig, axes = plt.subplots(nrows=n_in, ncols=n_out, figsize=(12, 6))

# Number of samples for each histogram
num_samples = 5000

# Iterate over rows and columns
for i in range(n_in):
    for j in range(n_out):
        # Get mean and standard deviation for specific indices
        mu = w_mean[i, j]
        sigma = tf.nn.softplus(w_std[i, j])
        
        # Sample epsilon from standard normal distribution
        epsilon = tf.random.normal(shape=(num_samples,))
        
        # reparameterization trick to sample from the distribution
        samples = mu + sigma * epsilon

        # Create x values for PDF plotting
        x = np.linspace(float(mu - 4*sigma), float(mu + 4*sigma), 1000)
        
        # Compute PDF manually 
        #pdf = (1. / (sigma * np.sqrt(2. * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

        # or compute pdf using tensorflow probability distribution
        pdf = tfp.distributions.Normal(loc=mu, scale=sigma).prob(x)

        # Plot the PDF
        ax = axes[i, j]
        ax.plot(x, pdf, label=f'$\mu = {mu:.3f}$\n$\sigma = {sigma:.3f}$', color='blue')

        # Plot the histogram of samples
        ax.hist(samples.numpy(), bins=100, density=True, alpha=0.6, color='orange')

        # Draw a vertical line at the mean
        ax.axvline(mu, color='r', linestyle='--')

        ax.set_title(f'PDF and Samples ({i},{j})')
        ax.set_xlabel('Value')
        ax.set_ylabel('Density')
        ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
def plot_histograms(plot_data, titles, bins = 500, nrows = 2, ncols = 2, figsize = (12, 6)):

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

    # Loop over the axes to plot each histogram
    for ax, data, title in zip(axes.flatten(), plot_data, titles):
        ax.hist(data, bins= bins, density=False)
        ax.set_title(title)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

softplus = lambda x: np.log(1 + np.exp(x))

n_in, n_out = [1000, 1000]  
xavier_values = xavier(n_in, n_out)
xavier_values = xavier_values.numpy()

plot_data = [
    xavier_values.flatten()[:,None],
    softplus(xavier_values.flatten()[:,None]),
    softplus(xavier_values.flatten()[:,None]-5),
    softplus(xavier_values.flatten()[:,None]-6)
]

titles = ['Xavier', 'Softplus of Xavier', 'Softplus of Xavier - 5','Softplus of Xavier - 6']


plot_histograms(plot_data, titles)

# Loss

In a BNN the goal is to compute the posterior distribution $p(\mathbf{w}| \mathbf{D})$, which represents our updated belief about the weights $\mathbf{w}$ after observing the data $\mathbf{D}$.

$$p(\mathbf{w}| \mathbf{D}) =  \frac{p(\mathbf{D}|\mathbf{w}) \cdot P(\mathbf{w})}{p(\mathbf{D})} = \frac{p(\mathbf{D}|\mathbf{w}) \cdot p(\mathbf{w})}{\int p(\mathbf{D}|\mathbf{w}')p(\mathbf{w}')d\mathbf{w}'}$$


Computing the marginal likelihood, $p(\mathbf{D})$, is particularly challenging for continuous weights as it requires integrating over the entire domain of the weights to account for all possible configurations. The integral in the denominator, $\int p(\mathbf{D}|\mathbf{w}')p(\mathbf{w}')d\mathbf{w}'$, represents the summation of probabilities over the entire continuous domain of the latent variable $\mathbf{w}$, rendering the computation of this value intractable. The same ideas would be applied also for the bias terms.

To address this issue, **variational inference** can be employed. This method approximates the true posterior distribution with a simpler, tractable distribution. The parameters of this approximating distribution are optimized to minimize the divergence between it and the true posterior. The measure used for this divergence is the **Kullback-Leibler (KL) divergence**, that will be explained in the following.


## Kullback-Leibler (KL) Divergence (Relative Entropy)

The Kullback-Leibler (KL) divergence is a measure that quantifies the difference or "distance" between two distributions. It is a powerful tool for comparing and contrasting distributions, which is particularly useful in areas such as information theory, machine learning, and statistical inference. Given two probability distributions, $p(x)$ and $q(x)$, the KL divergence can be defined using the following formula:

$$D_{KL}(p||q) =\mathbb{E}\bigg[\log{\bigg(\frac{p(x)}{q(x)}\bigg)}\bigg] $$

In the discrete case, the KL divergence is computed as:

$$D_{KL}(p||q) = \sum_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

For continuous probability distributions, the KL divergence is given by the integral:

$$D_{KL}(p||q) = \int_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

It is important to note that the KL divergence is not symmetric, meaning that $D_{KL}(p||q) \neq D_{KL}(q||p)$. The KL divergence can be interpreted as the expected logarithmic difference between the two probability distributions, with the expectation taken over the values of $x$ according to the distribution $p(x)$. In essence, the KL divergence quantifies the amount of information lost when we approximate the true distribution $p(x)$ using the distribution $q(x)$.

**Example 1 - Discrete**

Consider the set $w \in {0:Bad, 1:Good}$ representing the weather, and suppose that the weather being good or bad follows a Bernoulli distribution $w \sim \textbf{Bernoulli}(\theta)$. In a hypothetical scenario, two people are arguing about the chance of the weather being good. The first person says that $\theta_A = 0.8$, and the second person says that $\theta_B = 0.7$. How far apart are their opinions (distributions)?

For a Bernoulli distribution, it can be thought of as a model for the set of possible outcomes of any single experiment that asks a yes–no question. The Bernoulli distribution can be written as

$$\textbf{Bernoulli}(w,\theta) = \theta^w(1-\theta)^{1-w}$$

and the KL divergence as

$$D_{KL}(p||q) = \sum^{1}_{w=0} p(w)\log{\bigg(\frac{p(w)}{q(\mathbf{w})}\bigg)}$$

where the probabilities are $p(w) = \theta_A^w(1-\theta_A)^{1-w}$ and $q(\mathbf{w}) = \theta_B^w(1-\theta_B)^{1-w}$ . substituting in the Kl divergence:

$$D_{KL}(p||q) = \sum^{1}_{w=0} \theta_A^w(1-\theta_A)^{1-w}\log{\bigg(\frac{\theta_A^w(1-\theta_A)^{1-w}}{\theta_B^w(1-\theta_B)^{1-w}}\bigg)}$$

Now, let's evaluate the sum for $w=0$ and $w=1$:

$$D_{KL}(p||q) = (1-\theta_A)\log{\bigg(\frac{1-\theta_A}{1-\theta_B}\bigg)} + \theta_A\log{\bigg(\frac{\theta_A}{\theta_B}\bigg)}$$

Now, we can plug in the values $\theta_A = 0.8$ and $\theta_B = 0.7$ to compute the KL divergence:

$$D_{KL}(p||q) \approx 0.02573$$

This value represents the distance between the two opinions (distributions) held by the two people.

In [ ]:
weather_A = tfp.distributions.Bernoulli(probs = 0.8)
weather_B = tfp.distributions.Bernoulli(probs = 0.7)

print('D(p||q)=', float(tfp.distributions.kl_divergence(weather_A, weather_B)))

**Example 2 - Continous**

Suppose we're studying the height of adult men in two countries: Country A and Country B.
The height of adult men in Country A follows a Gaussian distribution with mean 175 cm and standard deviation 10 cm. The height of adult men in Country B follows a Gaussian distribution with mean 170 cm and standard deviation 15 cm.

we can use the Kullback-Leibler (KL) divergence to measure the difference between these two distributions. 

First, recall that the KL divergence between two Gaussian distributions $p(x)\sim \mathcal{N}(\mu_p, \sigma_p )$ and $q(x)\sim \mathcal{N}(\mu_q,\sigma_q )$ is 

$$D_{KL}(p||q) = \int_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

we can substitute their probability density functions into this integral and simplify the result. A probability density function $f(x) \sim \mathcal{N}(\mu,\sigma )$ for a Gaussian distribution with mean $\mu$ and standard deviation $\sigma$ is given by

$$f(x) = \frac{1}{\sqrt{2\pi \sigma}} \exp\bigg(-\frac{(x-\mu)^2}{2 \sigma^2}\bigg)$$

Substituting the expressions for $p(x) \sim \mathcal{N}(\mu_p,\sigma_p )$ and $q(x)\sim \mathcal{N}(\mu_q,\sigma_q )$  into the KL divergence integral and simplifying yields the formula:

$$D_{KL}(p||q) = \log\left(\frac{\sigma_q}{\sigma_p}\right) + \frac{\sigma_p^2 + (\mu_p - \mu_q)^2}{2\sigma_q^2} - \frac{1}{2}$$

Here, $p(x) \sim \mathcal{N}(175.3,7.1 )$ and $q(x)\sim \mathcal{N}(170.8,6.3)$. Substituting these values into the formula for the KL divergence gives:

$$D_{KL}(P||Q) = \log\left(\frac{6.3}{7.1}\right) + \frac{(7.1)^2 +(175.3−170.8)^2 
}{2(6.3))^2} - \frac{1}{2} \approx 0.183$$

$D_{KL}(P||Q) = 0.183$ indicate the amount of information lost when the distribution for Country B is used to approximate the distribution for Country A. This gives us a notion of how distance this two distribution are from each other.


In [ ]:
mu_p, sigma_p = 175, 10
mu_q, sigma_q = 170, 15

# distributions
dist_p = tfp.distributions.Normal(loc=mu_p, scale=sigma_p)
dist_q = tfp.distributions.Normal(loc=mu_q, scale=sigma_q)

# KL divergence
kl_divergence = tfp.distributions.kl_divergence(dist_p, dist_q)
print('D(p||q)=', float(kl_divergence))

Let's now examine a very simple example to help us understand the mechanics and foundational concept behind minimizing KL divergence to align two distributions.

**Naive Example 3 - Variational Inference idea**

Imagine we do not know the true distribution of adult men's heights in Country B, yet we seek to estimate it. We are tasked with adjusting the parameters of a variational distribution, denoted as $\mu_b$ and $\sigma_b$, which expresses our belief about Country B. Our goal is to construct an approximating distribution so that it resembles the established distribution of Country A. In Country A, the heights of adult men adhere to a Gaussian distribution, characterized by a mean $\mu_a = 175$ cm and a standard deviation $\sigma_a = 10$ cm.

By minimizing the KL divergence between the variational distribution and the true distribution of Country A, we aim to make the two distributions more similar and reduce the information loss.

In [ ]:
# Given parameters for distribution of Country A
mu_a = 175  
sigma_a = 10  

# Initialize parameters for distribution of Country B 
mu_b = tf.Variable(170.0)  # initial mean of Country B
sigma_b = tf.Variable(15.0)  # initial standard deviation of Country B

# Define the distributions
dist_a = tfp.distributions.Normal(loc=mu_a, scale=sigma_a)
dist_b = lambda: tfp.distributions.Normal(loc=mu_b, scale=tf.abs(sigma_b))  # ensure scale is positive

# Define the optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Optimization loop to minimize KL divergence
for step in range(1000):
    with tf.GradientTape() as tape:
        kl_divergence = tfp.distributions.kl_divergence(dist_a, dist_b())
    gradients = tape.gradient(kl_divergence, [mu_b, sigma_b])
    optimizer.apply_gradients(zip(gradients, [mu_b, sigma_b]))
    if step % 100 == 0:
        print(f'Step {step}, KL Divergence: {kl_divergence.numpy()}, mu_b: {mu_b.numpy()}, sigma_b: {sigma_b.numpy()}')

print('Optimized parameters for Country B distribution:')
print('mu_b:', mu_b.numpy())
print('sigma_b:', sigma_b.numpy())

In this scenario, we are applying the foundational idea that leads to variational inference. We employ an approximating distribution and minimize the KL divergence to closely resemble a target distribution we desire to model. This concept takes a central role in Variational Inference, particularly useful when faced with intractable true distributions, especially in Bayesian modeling where the objective is to estimate a posterior distribution relying on observed data and prior beliefs.

From what we observe, the Kullback-Leibler (KL) divergence acts as a fundamental connector in the approximation process within variational inference. It quantifies the difference or "distance" between two distributions, offering a method to assess the accuracy with which the approximating distribution mirrors the true posterior in a Bayesian model. The concept of variational inference and the role of KL divergence will be explored in more detail in the following section.

## Evidence Lower Bound (ELBO)


Variational inference is a powerful technique in Bayesian modeling used to approximate complex posterior distributions. The primary challenge in Bayesian inference is computing the posterior distribution $p(\mathbf{w}| \mathbf{D})$, which involves the intractable integral over the marginal likelihood $p(\mathbf{D})$. Variational inference tackles this by proposing a simpler, parametrized distribution $q(\mathbf{w})$, the variational distribution, to approximate the true posterior.

In the context of neural networks, during the feedforward step, we deal with weights and biases that are typically learned. However, in a Bayesian neural network (BNN), these weights and biases are viewed as random variables. Rather than having fixed values, these variables are often sampled from independent Gaussian distributions, making the assumption that the learnable parameters are uncorrelated. The Gaussian distributions for each of these parameters are defined by their mean and variance: $\mu_w$, $\sigma_w$ for weights, and $\mu_b$, $\sigma_b$ for biases.

One important simplification in variational inference is the mean field approach. Here, each parameter is assumed to have its own independent variational distribution. This implies that each weight and bias is treated independently, each governed by its distinct Gaussian distribution. While this simplification makes the optimization more tractable, it will not account for correlations between parameters.

After sampling from these Gaussian distributions and executing the feedforward step, the learned mean and variance parameters define the variational distributions for both weights $q(\mathbf{w})$ and biases $q(\mathbf{b})$. Although various distributions can be chosen for this purpose, multidimensional Gaussian distributions are frequently used in BNNs:


For the weights:
$$q(\mathbf{w}) = \frac{1}{(2\pi)^{\frac{mn}{2}}|{\Sigma}_w|^{1/2}} \exp\left(-\frac{1}{2} (\mathbf{w} - {\mu}_w)^T {\Sigma}_w^{-1} (\mathbf{w} - {\mu}_w)\right),$$


For the biases:
$$q(\mathbf{b}) = \frac{1}{(2\pi)^{n/2}|{\Sigma}_b|^{1/2}} \exp\left(-\frac{1}{2} (\mathbf{b} - {\mu}_b)^T {\Sigma}_b^{-1} (\mathbf{b} - {\mu}_b)\right),$$

Here, ${\Sigma}_w$ and ${\Sigma}_b$ are diagonal covariance matrices for weights and biases respectively. These matrices are diagonal, meaning that only the elements on the main diagonal are non-zero. These non-zero elements are the variances of the weights and biases, represented by $\sigma_w$ and $\sigma_b$ respectively. We can calculate the covariance matrices ${\Sigma}_w$ and ${\Sigma}_b$ by squaring each element of ${\sigma}_w$ and ${\sigma}_b$ respectively:

$$
{\Sigma}_w =
\left( \begin{array}{cccc}
\sigma_{11}^2 & 0 & \cdots & 0\\
0 & \sigma_{12}^2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \sigma_{mn}^2
\end{array} \right),~~~~~~
{\Sigma}_b =
\left( \begin{array}{cccc}
\sigma_{1}^2 & 0 & \cdots & 0\\
0 & \sigma_{2}^2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \sigma_{n}^2
\end{array} \right).
$$

Where $m$ is the number of rows in the weight matrix, representing the number of input features or connections from the previous layer and $n$ is the number of columns in the weight matrix, representing the number of neurons in the current layer, and also the dimensionality of the bias vector.


In this setup, the task is to optimize the parameters of the variational distributions ($\mu_w$, $\sigma_w$, $\mu_b$, $\sigma_b$) such that $q(\mathbf{w})$ and $q(\mathbf{b})$ closely approximate the true posterior distributions of weights and biases given the data, i.e., $p(\mathbf{w}| \mathbf{D})$ and $p(\mathbf{b}| \mathbf{D})$. We aim to demonstrate that the KL-divergence can be decomposed into a term known as the ELBO and another constant term. These can be minimized to bring the variational distribution closer to the true posterior. By doing so, we can make Bayesian inferences about the weights and biases of the neural network, incorporating uncertainty in our model.

To address this, we utilize the continuous KL-divergence to find a distribution $q(\mathbf{w})$ that closely approximates the true posterior distribution $p(\mathbf{w}| \mathbf{D})$. The goal is to minimize the KL-divergence to ensure the most accurate approximation possible.

$$D_{KL}(q(\mathbf{w})||p(\mathbf{w}|\mathbf{D})) = \int q(\mathbf{w'})\log{\bigg(\frac{q(\mathbf{w'})}{p(\mathbf{w'}| \mathbf{D})}\bigg)}d\mathbf{w'}$$

It may seem unusual that we choose $q(\mathbf{w'})$ instead of $p(\mathbf{w'}| \mathbf{D})$ in the integrand, but this is due to the lack of direct access to the true posterior distribution.  Thus we are using $q(\mathbf{w'})$ to approximate the intractable true posterior $p(\mathbf{w'}| \mathbf{D})$, so it’s natural to use $q(\mathbf{w'})$ in the computations.

Substituting Bayes' formula,

$$p(\mathbf{w'}| \mathbf{D}) = \frac{p(\mathbf{D}|\mathbf{w'})p(\mathbf{w'})}{p(\mathbf{D})}, $$ 

into the KL divergence gives:

$$D_{KL}(q(\mathbf{w})||p(\mathbf{w}|\mathbf{D})) = \int q(\mathbf{w'})\log{\bigg(\frac{q(\mathbf{w'})p(\mathbf{D})}{p(\mathbf{D}|\mathbf{w'})p(\mathbf{w'})}\bigg)}d\mathbf{w'}$$

We have information about the joint probability distribution $p(\mathbf{D}|\mathbf{w'})p(\mathbf{w'})$, but not about the marginal distribution $p(\mathbf{D})$. Rearranging the integral in terms of known and unknown probabilities and simplifying the notation of the KL-divergence:

$$D_{KL}(q||p) = \int q(\mathbf{w'})\log{\bigg(\frac{q(\mathbf{w'})}{p(\mathbf{D}|\mathbf{w'})p(\mathbf{w'})}\bigg)}d\mathbf{w'} + \log{p(\mathbf{D})}$$

Now, we aim to express this KL-divergence as a sum of expectations. Given a continuous probability distribution $q(\mathbf{w})$ for a random variable $\mathbf{w}$, and a function of this variable denoted as $g(\mathbf{w})$, the expectation value of $g$ with respect to the distribution $q(\mathbf{w})$ is calculated by:

$$\mathbb{E}_{\mathbf{w} \sim q}[g(\mathbf{w})] = \int g(\mathbf{w})q(\mathbf{w})d\mathbf{w}$$

With this, we can reinterpret the terms in the KL-divergence as expectation values, where the probabilities under the distribution $q(\mathbf{w})$ serve as the weights:

$$D_{KL}(q||p) = \mathbb{E}_{\mathbf{w} \sim q}\bigg[\log{\frac{q(\mathbf{w})}{p(\mathbf{D}|\mathbf{w})p(\mathbf{w})}}\bigg] + \log{p(\mathbf{D})}$$

Since $p(\mathbf{D})$ is not dependent on $\mathbf{w}$, the expectation $\mathbb{E}_{\mathbf{w} \sim q}[\log{p(\mathbf{D})}]$ simplifies to $\log{p(\mathbf{D})}$. Reversing the fraction in the first expectation and introducing a minus sign in the logarithm yields:

$$D_{KL}(q||p) = -\mathbb{E}_{\mathbf{w} \sim q}\bigg[\log{\frac{p(\mathbf{D}|\mathbf{w})p(\mathbf{w})}{q(\mathbf{w})}}\bigg] + \log{p(\mathbf{D})}$$

Let's denote the first expectation as the Evidence Lower Bound $\text{ELBO}(q(\mathbf{w})) = \mathbb{E}_{\mathbf{w} \sim q}\bigg[\log{\frac{p(\mathbf{D}|\mathbf{w})p(\mathbf{w})}{q(\mathbf{w})}}\bigg]$, and finally we can express the KL-divergence as:

$$D_{KL}(q(\mathbf{w})||p(\mathbf{w}|\mathbf{D})) = -\text{ELBO}(q(\mathbf{w})) + \log{p(\mathbf{D})}$$


We know that the KL-divergence must be non-negative , $D_{KL}(q||p) \geq 0$, as it measures the information loss between the true posterior distribution $p(\mathbf{w}| \mathbf{D})$ and the variational distribution $q(\mathbf{w})$. The term $\log{p(\mathbf{D})}$ is constant for a given input data. Given the properties of logarithms and since $0 < p(\mathbf{D}) \leq 1$, the log term must be $\log{p(\mathbf{D})} \leq 0$ .

Now, consider the Evidence Lower Bound (ELBO). As its definition, the relationship between ELBO, the log marginal likelihood, and the KL-divergence can be  expressed as in the following inequality:

$$-\text{ELBO}(q(\mathbf{w})) + \log{p(\mathbf{D})}\geq 0$$

Re-arranging terms, we observe:

$$\text{ELBO}(q(\mathbf{w})) \leq \log{p(\mathbf{D})}$$

This inequality is a clear indication that the ELBO serves as a lower bound to the log marginal likelihood, $\log{p(\mathbf{D})}$, hence the name Evidence Lower Bound.

Note that maximizing the ELBO is equivalent to minimizing the KL-divergence between the variational distribution $q(\mathbf{w})$ and the true posterior $p(\mathbf{w}|\mathbf{D})$. The idea behind this is that by minimizing the information loss represented by the KL-divergence, we are essentially approximate the true posterior by the variational distribution.

$$\text{ELBO}(q(\mathbf{w}))= -D_{KL}(q(\mathbf{w})||p(\mathbf{w}|\mathbf{D})) + \log{p(\mathbf{D})}$$

In [ ]:
mixture_probs = [0.3, 0.4, 0.3]
mus = [-1.3, 2.2, 4.0]
sigmas = [2.3, 1.5, 4.4]

# Generate an array of latent variables (weights)
w = tf.linspace(-5, 5, 100)
w = tf.cast(w, tf.float32)

# Create a true posterior distribution 
true_posterior = tfp.distributions.MixtureSameFamily(
    mixture_distribution=tfp.distributions.Categorical(probs=mixture_probs),
    components_distribution=tfp.distributions.Normal(loc=mus, scale=sigmas)
)

# Compute the density values of the true posterior for each 'w' value.
prob_values_true = true_posterior.prob(w)

mu_range = tf.linspace(-5, 5, 101)  
mu_range = tf.cast(mu_range, tf.float32)  

sigma_range = tf.linspace(0.1, 5, 50)  
sigma_range = tf.cast(sigma_range, tf.float32)  

# Initialize KL_VALUES with zeros
KL_VALUES = tf.zeros((len(sigma_range), len(mu_range)))

NUM_SAMPLES = 100

# Calculate KL divergences
for i, sigma_val in enumerate(sigma_range):
    for j, mu_val in enumerate(mu_range):
        variational_distribution = tfp.distributions.Normal(loc=mu_val, scale=sigma_val)

        samples = variational_distribution.sample(NUM_SAMPLES)
        log_p = true_posterior.log_prob(samples)
        log_q = variational_distribution.log_prob(samples)
        kl_estimate = tf.reduce_mean(log_q - log_p)
        
        KL_VALUES = tf.tensor_scatter_nd_update(KL_VALUES, [[i, j]], [kl_estimate])

# Convert KL_VALUES to a numpy array
KL_VALUES = KL_VALUES.numpy()

In [ ]:
# === PLOT ANIMATION FUNCTION ===
# This function creates and returns a plotly figure to visualize the distribution
def generate_figure(mu_approx, sigma_approx):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=w, y=prob_values_true, mode="lines", name="True Posterior"))
    
    variational_distribution = tfp.distributions.Normal(loc=mu_approx, scale=sigma_approx)
    var_dist_prob = variational_distribution.prob(w).numpy()

    fig.add_trace(go.Scatter(x=w, y=var_dist_prob, mode="lines", name="Variational Distribution"))

    fig.update_layout(
        xaxis={
            "gridcolor": "grey",
            "range": [-5.1, 5.1],
        },
        yaxis={
            "gridcolor": "grey",
            "range": [-0.1, 0.5],
        },
        margin=dict(l=10, r=10, t=10, b=10),
        width=700,
        height=300,
    )

    return fig

# === LATEX STRING FUNCTION ===
# This function creates and returns a formatted LaTeX string based on provided parameters
def generate_elbo_str(mu_approx, sigma_approx):
    mu_entry_id = int(10.0 * (mu_approx + 5.0)) - 1
    sigma_entry_id = int(10.0 * (sigma_approx - 0.1)) - 1
    
    evidence = - tf.reduce_min(KL_VALUES)
    kl = KL_VALUES[sigma_entry_id, mu_entry_id]
    elbo = - kl + evidence
    elbo_str = f"${{{elbo:.3f}}}_{{\\text{{ELBO}}}} =  - {{{kl:.3f}}}_{{\\text{{KL}}}} + {{{evidence:.3f}}}_{{\\text{{Evidence}}}}$"
    elbo_ltx = pn.pane.LaTeX(elbo_str, sizing_mode='stretch_width', styles= {'font-size':'18pt'}) 
    
    return elbo_ltx

# === WIDGET CONFIGURATION ===
# Define and configure widgets (sliders) for mu and sigma
mu_slider = pn.widgets.FloatSlider(name='mu:', start=-5.0, end=5.0, step=0.1, value=0.0)
sigma_slider = pn.widgets.FloatSlider(name='sigma:', start=0.1, end=5.0, step=0.1, value=1.0)

# Additional configuration for sliders
slider_width = 300
mu_slider.width = slider_width
sigma_slider.width = slider_width
mu_slider.align = 'center'
sigma_slider.align = 'center'

# LaTeX string for the ELBO equation
latex_str = r'$$\text{ELBO}(q(\mathbf{w}))= -D_{KL}(q||p) + \log{p(\mathbf{D})}$$'
latex_panel = pn.pane.LaTeX(latex_str, sizing_mode='stretch_width', styles= {'font-size':'18pt'})

# Binding functions with widgets to allow for dynamic updates
fig_pane = pn.bind(generate_figure, mu_approx=mu_slider, sigma_approx=sigma_slider)
elbo_pane = pn.bind(generate_elbo_str, mu_approx=mu_slider, sigma_approx=sigma_slider)

# Organizing layout elements
latex_panel =  pn.Row(pn.Spacer(width=250), latex_panel, pn.Spacer(width=400))
mu_slider = pn.Row(pn.Spacer(width=400), mu_slider, pn.Spacer(width=400))
sigma_slider = pn.Row(pn.Spacer(width=400), sigma_slider, pn.Spacer(width=400))
elbo_pane = pn.Row(pn.Spacer(width=270), elbo_pane, pn.Spacer(width=400))
fig_pane = pn.Row(pn.Spacer(width=200), fig_pane, pn.Spacer(width=400))

# === FINAL DASHBOARD ===
# Combine all elements into a single layout and make it serveable
dashboard = pn.Column(
    latex_panel,
    elbo_pane,
    fig_pane,
    mu_slider,
    sigma_slider
)
dashboard.servable()

From our graphical animation,  we can observe that maximizing the ELBO is equivalent  to minimizing the KL-divergence between the variational distribution $q(\mathbf{w})$ and the true posterior $p(\mathbf{w}∣\mathbf{D})$. This observation underscores a fundamental idea: by reducing the KL-divergence, which quantifies the information differences between the distributions, we're  aligning our variational distribution more closely with the true posterior. In essence, the ELBO captures the balance between the log marginal likelihood (evidence) $\log {q(\mathbf{D})}$ and the KL-divergence  $D_{KL}(q||p)$.

# Backpropagation Step

Once the loss function is defined and calculated after the feedforward step, the backpropagation process begins. This step computes the gradient of the loss with respect to each weight by propagating the gradient backward through the network. The goal is to adjust the weights and biases in a way that minimizes the loss. In traditional Neural Networks, standard loss functions like the Mean Squared Error or Cross-Entropy are used. However, as we discussed before, in a BNN, we aim to maximize the ELBO to find weights and biases that best approximate two distributions: the unknown posterior distribution $p(\mathbf{w}|\mathbf{D})$ and a variational distribution $q(\mathbf{w})$. The ELBO, or Evidence Lower Bound, in terms of the KL-divergence is:

$$\text{ELBO}(q(\mathbf{w}))= -D_{KL}(q(\mathbf{w})||p(\mathbf{w}|\mathbf{D})) + \log{p(\mathbf{D})}$$

Maximizing the ELBO effectively minimizes the Kullback-Leibler (KL) divergence between the variational distribution  and the true posterior, thereby making  $q(\mathbf{w})$ a better approximation of $p(\mathbf{w}|\mathbf{D})$. The constant term in the equation represents the log-likelihood of the data, denoted as $\log{p(\mathbf{D})}$, and remains constant throughout the optimization process.

To better illustrate this step, consider a fully connected network with three layers for the backpropagation process:

<center><img src="Images/bnn_bp.svg" width="800" height="800"></center>


In the backpropagation step between the hidden layer and the input layer, each connection represents a weight $w_{ij}$, and each weight has its own associated variational distribution $q(w_{ij})$. These distributions are iteratively refined as we maximize the ELBO. At the end of this process, we obtain variational distributions that closely approximate the true posterior distributions of the weights, given the observed data $\mathbf{D}$.

While we have focused on weights, it's worth mentioning that biases are also part of this process and follow a similar logic; they are omitted here for brevity. Now, consider a multivariate Gaussian as the general form for variational distribution $q(\mathbf{w})$ for the weights between the hidden and output layers:

$$q(\mathbf{w}) = \frac{1}{(2\pi)^{\frac{mn}{2}}|{\Sigma}_w|^{1/2}} \exp\left(-\frac{1}{2} (\mathbf{w} - {\mu}_w)^T {\Sigma}_w^{-1} (\mathbf{w} - {\mu}_w)\right),$$

Here, $\Sigma_w$ represents  the covariance matrix of the weights and $\mathbf{w}$ is the matrix with the weights values. For dependent variables $\mathbf{w}$, the full covariance matrix $\Sigma_w$ is:

$$
\Sigma_w = \begin{pmatrix}
\sigma_1^2 & \rho_{12} & \cdots & \rho_{1,n} \\
\rho_{21} & \sigma_2^2 & \cdots & \rho_{2,n} \\
\vdots  & \vdots  & \ddots & \vdots  \\
\rho_{m,1} & \rho_{m,2} & \cdots & \sigma_{mn}^2 
\end{pmatrix},~~~~
\mathbf{w} =
\left( \begin{array}{cccc}
w_{11} & \cdots & w_{1n}\\
\vdots & \ddots & \vdots\\
w_{m1} & \cdots &w_{mn}
\end{array} \right).
$$

Here, the off-diagonal elements $\rho_{ij}$ for $ i\neq j$ indicate the covariance between the different weights, and the diagonal elements $\sigma_{ii}^2 $ are the variances of each weight $w_{ij}$ in the matrix $\mathbf{w}$. 

Using this multivariate Gaussian results in a more complex model that can potentially capture intricate correlations between weights, thereby improving the model's accuracy, especially in the context of variational inference. However, this introduces complexity into the optimization process, making it potentially intractable. This is because of the high-dimensional nature of the ELBO function, where each dimension corresponds to an element in the covariance matrix $\Sigma_w$.

To contour the computational complexity, the mean field approach to variational inference offers an approximation to handle this issue. This technique simplifies the optimization of the ELBO by allowing us to treat each variational distribution over the weights as an independent entity. In essence, the mean field approach decouples the elements of the covariance matrix $\Sigma_w$, reducing it to a diagonal matrix where each diagonal entry represents a univariate Gaussian distribution corresponding to the variance of each individual weight in the matrix. By adopting the mean field approach, we consider the following simplified covariance matrix:

$$
{\Sigma}_w =
\left( \begin{array}{cccc}
\sigma_{11}^2 & 0 & \cdots & 0\\
0 & \sigma_{12}^2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \sigma_{mn}^2
\end{array} \right),
$$

Where $\mathbf{w}$ remains as previously defined. This diagonal matrix not only makes the optimization tractable but also significantly reduces the computational complexity, making it a practical choice for large-scale Bayesian Neural Networks as we will see.

Considering this diagonal covariance matrix ${\Sigma}_w$, the variational multivariate Gaussian simplifies to a product of independent gaussian distributions of each weight:

$$q(\mathbf{w}) = \prod^{m}_{i}\prod^{n}_{j} q(w_{ij})$$